In [1]:
using Plots, Interact
gr()

Plots.GRBackend()

In [2]:
include("discs_in_box.jl")

INFO: Recompiling stale cache file /Users/dpsanders/.julia/lib/v0.6/Distributions.ji for module Distributions.


dynamics

In [9]:
w, h = 1, 1
r = 0.24

@time times, positions, velocities, collision_types = dynamics(w, h, r, 1000000);

  2.949637 seconds (14.81 M allocations: 1.063 GiB, 75.45% gc time)


In [4]:
showall(norm.(velocities)[1:100])

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]

In [5]:
showall(collision_types[1:100])

[-1, 5, -1, 3, 5, 3, -1, 5, -4, 3, 2, 5, -1, 5, 3, 5, -2, 5, 4, -1, 5, -1, 3, 2, 5, -1, -4, 5, -1, 2, 3, 5, -4, 2, -1, 3, 5, 3, 5, 4, -1, 5, -1, 3, 5, -2, -1, 3, 5, -4, 2, 5, 3, -1, 2, -4, -3, 5, 1, 5, 2, -4, 5, -3, 5, -3, 1, -2, 5, -3, 5, 1, 5, -3, 4, 5, 1, -2, 5, -3, 5, 1, 4, 5, -3, -2, -1, 4, 5, 4, 5, -2, 5, 4, 5, -2, 5, 4, 1, 5]

The function `dynamics` returns data on the collisions that have occurred.

In [4]:
function draw_disc!(x, y, r)
    θs = [0:0.1:2π; 0]
    xs = [x + r*cos(θ) for θ in θs]
    ys = [y + r*sin(θ) for θ in θs]
    
    plot!(xs, ys)
end

draw_disc! (generic function with 1 method)

In [10]:
@manipulate for i in slider(1:length(positions), value=1)
    x1, y1, x2, y2 = positions[i]
    
    a = w/2
    b = h/2
    
    plot([-a, a, a, -a, -a], [-b, -b, b, b, -b], aspectratio=1, leg=false, xlim=(-0.5,0.5), ylim=(-0.5,0.5))
    draw_disc!(x1, y1, r) 
    draw_disc!(x2, y2, r) 
end

Interact.Slider{Int64}(9: "input-3" = 1 Int64 , "", 1, 1:1000000, "horizontal", true, "d", true)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -0.50 
 
 
 -0.25 
 
 
 0.00 
 
 
 0.25 
 
 
 0.50 
 
 
 -0.50 
 
 
 -0.25 
 
 
 0.00 
 
 
 0.25 
 
 
 0.50

# Hopping

The data from the simulation directly gives disc and wall collision times.
The other times that we require are hopping times, both horizontal, when $x_1 = x_2$, and vertical, when $y_1 = y_2$. 

Vertical hops can be detected from the data when $y_2 - y_1$ changes sign between two consecutive collisions.
Using the relative velocity $v_2 - v_1$, the vertical hopping time when $y_2 - y_1 = 0$ can be recovered.

In [18]:
"""
Calculate the times at which horizontal hops occur
"""
function horizontal_hopping_times(times, positions, velocities)
    Δxs = [x[3] - x[1] for x in positions]  # x_2 - x_1
    Δus = [v[3] - v[1] for v in velocities]  # u_2 - u_1;
    
    # indices where there is a hop between collisions i and i+1: 
    horiz_hop_indices = find( sign(Δxs[i]) != sign(Δxs[i+1]) for i in 1:length(positions)-1 );  
    
    # x + t*u = 0   so   t = -x/u
    horiz_hopping_times = times[horiz_hop_indices] - (Δxs[horiz_hop_indices] ./ Δus[horiz_hop_indices])
    
    return horiz_hopping_times
end

"""
Calculate the times at which horizontal hops occur
"""
function vertical_hopping_times(times, positions, velocities)
    Δys = [x[4] - x[2] for x in positions]  # y_2 - y_1
    Δvs = [v[4] - v[2] for v in velocities]  # v_2 - v_1;
    
    # indices where there is a hop between collisions i and i+1: 
    vert_hop_indices = find( sign(Δys[i]) != sign(Δys[i+1]) for i in 1:length(positions)-1 );  
    
    # x + t*u = 0   so   t = -x/u
    vert_hopping_times = times[vert_hop_indices] - (Δys[vert_hop_indices] ./ Δvs[vert_hop_indices])
    
    return vert_hopping_times
end

vertical_hopping_times

Analytical expression for horizontal hopping:

In [9]:
horiz_hop_analytical(a, b, r) = (3π / (2*√(2))) * (2*a^2*b^2 - 2π*a*b*r^2 + ((a+b)/3)*(2*r)^3 - r^4) / (a*√(2)*(b-r)^2)

disc_collision_analytical(a, b, r) = (3π / (2*√(2))) * (2*a^2*b^2 - 2π*a*b*r^2 + ((a+b)/3)*(2*r)^3 - r^4) / 

                                                                (2π*a*b*r - 4*(a+b)*r^2 + 2r^3 )

disc_collision_analytical (generic function with 1 method)

In [19]:
function simulation(w, h, num_collisions=10^5)
    
    num_horiz_hop_data = Float64[]
    exact_horiz_hop_data = Float64[]
    num_disc_collision_data = Float64[]
    exact_disc_collision_data = Float64[]
    
    rs = 0.005:0.005:0.23
    
    for r in rs
        print(r, " ")
        
        a = w/2 - r
        b = h/2 - r
    
        times, positions, velocities, collision_types = dynamics(w, h, r, num_collisions);
        
        
        horiz_hopping_times = horizontal_hopping_times(times, positions, velocities)

        push!(num_horiz_hop_data, mean(diff(horiz_hopping_times)))   # diff gives inter-hop times
        push!(exact_horiz_hop_data, horiz_hop_analytical(a, b, r))
        
        
        disc_collision_times = times[collision_types .== 5]
        push!(num_disc_collision_data, mean(diff(disc_collision_times)))   # diff gives inter-hop times
        push!(exact_disc_collision_data, disc_collision_analytical(a, b, r))

    end
    
    return rs, num_horiz_hop_data, exact_horiz_hop_data, num_disc_collision_data, exact_disc_collision_data
end

LoadError: [91msyntax: unexpected "}"[39m

In [11]:
w, h = 1.0, 1.0
rs, num_horiz_hop_data, exact_horiz_hop_data, num_disc_collision_data, exact_disc_collision_data = simulation(w, h);

0.005 0.01 0.015 0.02 0.025 0.03 0.035 0.04 0.045 0.05 0.055 0.06 0.065 0.07 0.075 0.08 0.085 0.09 0.095 0.1 0.105 0.11 0.115 0.12 0.125 0.13 0.135 0.14 0.145 0.15 0.155 0.16 0.165 0.17 0.175 0.18 0.185 0.19 0.195 0.2 0.205 0.21 0.215 0.22 0.225 0.23 

In [12]:
Plots.scatter(rs, num_disc_collision_data, m=:square, label="numerical")
plot!(rs, exact_disc_collision_data, label="exact")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.00 
 
 
 0.05 
 
 
 0.10 
 
 
 0.15 
 
 
 0.20 
 
 
 0 
 
 
 10 
 
 
 20 
 
 
 30 
 
 
 40 
 
 
 50 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 numerical 
 
 
 
 exact

In [13]:
plot(rs, num_horiz_hop_data, m=:square, label="numerical")
plot!(rs, exact_horiz_hop_data, m=:square, label="exact")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.00 
 
 
 0.05 
 
 
 0.10 
 
 
 0.15 
 
 
 0.20 
 
 
 2 
 
 
 3 
 
 
 4 
 
 
 5 
 
 
 6 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 numerical 
 
 
 
 
 
 exact

In [14]:
plot(rs, exact_horiz_hop_data./num_horiz_hop_data, m=:square)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.00 
 
 
 0.05 
 
 
 0.10 
 
 
 0.15 
 
 
 0.20 
 
 
 1.350 
 
 
 1.375 
 
 
 1.400 
 
 
 1.425 
 
 
 1.450 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 y1

In [15]:
histogram(diff(horiz_hopping_times))

LoadError: [91mUndefVarError: horiz_hopping_times not defined[39m

In [16]:
histogram(diff(horiz_hopping_times), normed=true)

LoadError: [91mUndefVarError: horiz_hopping_times not defined[39m

# Hitting right wall

In [29]:
V_exact(a, b, r) = 16a^2*b^2 - 16π*a*b*r^2 + (64/3)*(a+b)*r^3 - 8r^4

disc_1_hits_right_wall(a, b, r) = 8*a*b^2 - 4π*b*r^2 + (16/3) * r^3

disc_1_hits_right_wall (generic function with 1 method)

In [21]:
disc_1_hits_right_wall_times = times[collision_types .== 1]

8444-element Array{Float64,1}:
    1.98975
    2.95109
    3.19826
    3.27112
    3.75055
    3.8023 
    3.85744
    4.60162
    5.45908
    6.48873
    6.76875
    7.20229
    7.24794
    ⋮      
 8195.99   
 8196.62   
 8196.99   
 8197.31   
 8197.96   
 8205.74   
 8207.99   
 8208.16   
 8208.29   
 8209.39   
 8210.45   
 8210.82   

In [22]:
S = 2*π^2 
B = 4π / 3

factor = S / B

exact_mean_hitting_time(a, b, r) = factor * V_exact(a, b, r) / disc_1_hits_right_wall(a, b, r)

exact_mean_hitting_time (generic function with 1 method)

In [23]:
function simulation2(w, h, num_collisions=10^5)
    
    num_hitting_times = Float64[]
    exact_hitting_times = Float64[]
    
    rs = 0.005:0.005:0.23
    
    for r in rs
        print(r, " ")
        
        a = w/2 - r
        b = h/2 - r
    
        times, positions, velocities, collision_types = dynamics(w, h, r, num_collisions);
        
        disc_1_hits_right_wall_times = times[collision_types .== 1]

        push!(num_hitting_times, mean(diff(disc_1_hits_right_wall_times)))   # diff gives inter-hop times
        push!(exact_hitting_times, exact_mean_hitting_time(a, b, r))
        
        
    end
    
    return rs, num_hitting_times, exact_hitting_times
end

simulation2 (generic function with 2 methods)

In [24]:
rs, num_hitting_times, exact_hitting_times = simulation2(1.0, 1.0)

0.005 0.01 0.015 0.02 0.025 0.03 0.035 0.04 0.045 0.05 0.055 0.06 0.065 0.07 0.075 0.08 0.085 0.09 0.095 0.1 0.105 0.11 0.115 0.12 0.125 0.13 0.135 0.14 0.145 0.15 0.155 0.16 0.165 0.17 0.175 0.18 0.185 0.19 0.195 0.2 0.205 0.21 0.215 0.22 0.225 0.23 

(0.005:0.005:0.23, [4.67758, 4.58948, 4.49672, 4.43488, 4.44265, 4.41639, 4.36844, 4.30181, 4.20581, 4.21032  …  1.95438, 1.88366, 1.7964, 1.68128, 1.57659, 1.44244, 1.33599, 1.30018, 1.06917, 1.0123], [4.66453, 4.6152, 4.56441, 4.51217, 4.4585, 4.4034, 4.34688, 4.28895, 4.22961, 4.16888  …  1.98724, 1.88377, 1.77828, 1.67073, 1.56107, 1.44932, 1.33552, 1.21988, 1.10282, 0.985235])

In [25]:
plot(rs, num_hitting_times, m=:square, label="numerical")
plot!(rs, exact_hitting_times, m=:square, label="exact")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.00 
 
 
 0.05 
 
 
 0.10 
 
 
 0.15 
 
 
 0.20 
 
 
 1 
 
 
 2 
 
 
 3 
 
 
 4 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 numerical 
 
 
 
 
 
 exact

# Disc collisions

In [29]:
disc_collision_exact_area(a, b, r) = 16*π*a*b*r - 32*(a+b)*r^2 + 16*r^3 

exact_disc_collision(a, b, r) = factor * V_exact(a, b, r) / disc_collision_exact_area(a, b, r)

exact_disc_collision (generic function with 1 method)

In [30]:
function simulation_disc_collisions(w, h, num_collisions=10^5)
    
    num_times = Float64[]
    exact_times = Float64[]
    
    rs = 0.005:0.005:0.23
    
    for r in rs
        print(r, " ")
        
        a = w/2 - r
        b = h/2 - r
    
        times, positions, velocities, collision_types = dynamics(w, h, r, num_collisions);
        
        collision_times = times[collision_types .== 5]

        push!(num_times, mean(diff(collision_times)))   # diff gives inter-hop times
        push!(exact_times, exact_disc_collision(a, b, r))
        
        
    end
    
    return rs, num_times, exact_times
end

simulation_disc_collisions (generic function with 2 methods)

In [31]:
rs, num_disc_collision_times, exact_disc_collision_times = simulation_disc_collisions(w, h)

0.005 0.01 0.015 0.02 0.025 0.03 0.035 0.04 0.045 0.05 0.055 0.06 0.065 0.07 0.075 0.08 0.085 0.09 0.095 0.1 0.105 0.11 0.115 0.12 0.125 0.13 0.135 0.14 0.145 0.15 0.155 0.16 0.165 0.17 0.175 0.18 0.185 0.19 0.195 0.2 0.205 0.21 0.215 0.22 0.225 0.23 

(0.005:0.005:0.23, [52.3012, 25.9111, 17.0541, 12.5565, 10.0964, 8.41752, 7.11017, 6.0247, 5.34809, 4.79019  …  0.62033, 0.577382, 0.523786, 0.478765, 0.435009, 0.395654, 0.353614, 0.319724, 0.277916, 0.243282], [74.4391, 36.9232, 24.4074, 18.1415, 14.3756, 11.8597, 10.058, 8.70279, 7.64516, 6.79585  …  0.882501, 0.812518, 0.745294, 0.680651, 0.618443, 0.558563, 0.50095, 0.445609, 0.392653, 0.342378])

In [33]:
plot(rs, num_disc_collision_times, m=:square, label="numerical")
plot!(rs, exact_disc_collision_times, m=:square, label="exact")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.00 
 
 
 0.05 
 
 
 0.10 
 
 
 0.15 
 
 
 0.20 
 
 
 0 
 
 
 20 
 
 
 40 
 
 
 60 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 numerical 
 
 
 
 
 
 exact

In [35]:
plot(rs, exact_disc_collision_times./num_disc_collision_times, m=:square, label="ratio")


<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.00 
 
 
 0.05 
 
 
 0.10 
 
 
 0.15 
 
 
 0.20 
 
 
 1.40 
 
 
 1.41 
 
 
 1.42 
 
 
 1.43 
 
 
 1.44 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 ratio

Something to do with relative velocity?

## Vertical hopping

In [35]:
A_vert_hop_exact(a, b, r) = 8*b * sqrt(2)* (a-r)^2 

S = 2*π^2 
B = 4π / 3

factor = S / B

exact_vert_hopping(a, b, r) = factor * V_exact(a, b, r) / (2 * A_vert_hop_exact(a, b, r))
# el factor de 2 se debe a que se puede acceder desde dos lados al conjunto

exact_vert_hopping (generic function with 1 method)

In [36]:
function simulation_vert_hopping(w, h, num_collisions=10^5)
    
    num_times = Float64[]
    exact_times = Float64[]
    
    rs = 0.005:0.005:0.23
    
    for r in rs
        print(r, " ")
        
        a = w/2 - r
        b = h/2 - r
    
        times, positions, velocities, collision_types = dynamics(w, h, r, num_collisions);
        
        vert_hopping_times = vertical_hopping_times(times, positions, velocities)

        push!(num_times, mean(diff(vert_hopping_times)))   # diff gives inter-hop times
        push!(exact_times, exact_vert_hopping(a, b, r))
        
    end
    
    return rs, num_times, exact_times
end

simulation_vert_hopping (generic function with 2 methods)

In [37]:
rs, num_vert_hopping_times, exact_vert_hopping_times = simulation_vert_hopping(w, h)

0.005 0.01 0.015 0.02 0.025 0.03 0.035 0.04 0.045 0.05 0.055 0.06 0.065 0.07 0.075 0.08 0.085 0.09 0.095 0.1 0.105 0.11 0.115 0.12 0.125 0.13 0.135 0.14 0.145 0.15 0.155 0.16 0.165 0.17 0.175 0.18 0.185 0.19 0.195 0.2 0.205 0.21 0.215 0.22 0.225 0.23 

(0.005:0.005:0.23, [1.70211, 1.70849, 1.73376, 1.7546, 1.75107, 1.75934, 1.79558, 1.80619, 1.78078, 1.82043  …  2.46747, 2.46998, 2.53552, 2.69197, 2.65635, 2.87706, 3.05583, 3.35446, 3.65881, 4.35129], [1.68272, 1.69931, 1.71586, 1.73238, 1.74886, 1.76531, 1.78174, 1.79815, 1.81456, 1.83096  …  2.44723, 2.50425, 2.57218, 2.65492, 2.75835, 2.89172, 3.07038, 3.32167, 3.69898, 4.32082])

In [38]:
plot(rs, num_vert_hopping_times, m=:square, label="numerical")
plot!(rs, exact_vert_hopping_times, m=:square, label="exact")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.00 
 
 
 0.05 
 
 
 0.10 
 
 
 0.15 
 
 
 0.20 
 
 
 2.0 
 
 
 2.5 
 
 
 3.0 
 
 
 3.5 
 
 
 4.0 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 numerical 
 
 
 
 
 
 exact

El factor de 2 viene del hecho de que se puede acceder desde dos lados.

In [39]:
plot(rs, num_vert_hopping_times ./ exact_vert_hopping_times, m=:square, label="numerical")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.00 
 
 
 0.05 
 
 
 0.10 
 
 
 0.15 
 
 
 0.20 
 
 
 0.97 
 
 
 0.98 
 
 
 0.99 
 
 
 1.00 
 
 
 1.01 
 
 
 1.02 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 numerical

## Symbolics

In [8]:
using SymPy

In [10]:
@syms x y

(x, y)

  likely near /Users/dpsanders/.julia/v0.6/IJulia/src/kernel.jl:31
  likely near /Users/dpsanders/.julia/v0.6/IJulia/src/kernel.jl:31
  likely near /Users/dpsanders/.julia/v0.6/IJulia/src/kernel.jl:31
in jprint at /Users/dpsanders/.julia/v0.6/SymPy/src/display.jl


In [11]:
x

x

In [12]:
x*x

 2
x 

In [13]:
sin(x)

sin(x)

In [16]:
simplify(sin(x)^2 + cos(x)^2)

1